## Valid Node Coords

In [ ]:
import numpy as np

grid_shape = np.array([70, 70, 32])
R = 35
r = 3

node_center = (grid_shape[0]/2, grid_shape[1]/2)

In [ ]:
from itertools import product

node_coords = np.array(list(product(range(grid_shape[0] + 1),
                                    range(grid_shape[1] + 1))))

dist_from_center = np.linalg.norm(node_coords - node_center, axis=1)

valid_node_mask = (dist_from_center >= r+1) & (dist_from_center <= R)

valid_node_coords_2d = node_coords[valid_node_mask]
valid_node_coords_2d

## Valid Cell Coords

In [ ]:
from numba import njit


surrounding_nodes = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]]
)

@njit
def check_valid_cells(cell_coords, valid_node_coords, mask):
    # cell center 주위의 4개 node가 valid_node_coords에 모두 포함되어 있는지 확인
    for i, cell_coord in enumerate(cell_coords):
        surrounding_node_coords = cell_coord + surrounding_nodes
        flag = True
        for node_coord in surrounding_node_coords:
            x, y = node_coord
            valid = valid_node_coords[
                (valid_node_coords[:, 0] == x) & (valid_node_coords[:, 1] == y)
            ].any()

            if not valid:
                flag = False
                break
        mask[i] = flag

In [ ]:
cell_coords = np.array(list(product(range(grid_shape[0]),
                                    range(grid_shape[1]))))

valid_cell_mask = np.empty(cell_coords.shape[0], dtype=bool)
valid_cell_mask

check_valid_cells(cell_coords, valid_node_coords_2d, valid_cell_mask)

cell_coords[valid_cell_mask]

In [ ]:
valid_cells = np.array(list((x, y, z) for (x, y), z in product(
    cell_coords[valid_cell_mask],
    range(grid_shape[2])
)))
np.save('valid_cell_coords.npy', valid_cells)

## Fixed Field Boundary

- 기존 코드에서 자기장이 경계면에 반사되어 간섭을 일으키는 문제 발생
- 3가지 해결책
    - PML
    - Conductor
    - Boundary에 있는 field 값을 초기값으로 고정
- 이 중 3번째 방법을 위해 고정시킬 grid point들의 좌표를 구함
- [Valid Node Coords](#Valid-Node-Coords)와 같은 방법으로 구하지만, 경계면도 포함해야 하며,
- Z좌표의 최소, 최댓값을 갖는 부분도 포함한다.

### Fiexd E field
node에서 정의

In [ ]:
fixed_node_mask = (dist_from_center <= r+3) | (dist_from_center >= R-2)

fixed_node_coords_2d = node_coords[fixed_node_mask]
fixed_node_coords_2d

In [ ]:
fixed_xy = set((x, y, z) for (x, y), z
               in product(fixed_node_coords_2d,
                          range(grid_shape[2])))
fixed_z = set(product(range(grid_shape[0] + 1),
                      range(grid_shape[1] + 1),
                      (0, grid_shape[2])))  # lower and upper bound of z

In [ ]:
fixed_node_coords = np.array(list(fixed_xy | fixed_z))
np.save('constant_field_node_coords.npy', fixed_node_coords)

### Fixed B field
cell center에서 정의   
fixed_node_coords_2d로 모두 둘러싸인 cell을 fixed_center_coords_2d로 설정   
z좌표 최소, 최댓값 포함

In [ ]:
fixed_cell_mask = np.empty(cell_coords.shape[0], dtype=bool)

check_valid_cells(cell_coords, fixed_node_coords_2d, fixed_cell_mask)

fixed_cell_coords_2d = cell_coords[fixed_cell_mask]

In [ ]:
fixed_xy = set((x, y, z) for (x, y), z
               in product(fixed_cell_coords_2d,
                          range(grid_shape[2])))
fixed_z = set(product(range(grid_shape[0]),
                      range(grid_shape[1]),
                      (0, grid_shape[2]-1)))  # lower and upper bound of z

In [ ]:
fixed_center_coords = np.array(list(fixed_xy | fixed_z))
np.save('constant_field_center_coords.npy', fixed_center_coords)

## Plot

In [ ]:
cell_coords[valid_cell_mask]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(
    x=fixed_node_coords_2d[:, 0],
    y=fixed_node_coords_2d[:, 1],
    mode='markers',
    opacity=0.5,
    name='constant_field__node_coords'
))

fig.add_trace(go.Scatter(
    x=cell_coords[valid_cell_mask][:, 0] + 0.5,
    y=cell_coords[valid_cell_mask][:, 1] + 0.5,
    mode='markers',
    opacity=0.5,
    name='valid_cell_coords'
))

fig.add_trace(go.Scatter(
    x=fixed_cell_coords_2d[:, 0] + 0.5,
    y=fixed_cell_coords_2d[:, 1] + 0.5,
    mode='markers',
    opacity=0.5,
    name='constant_field_center_coords'
))


fig.update_layout(height=800, width=800)
fig